## Objective

We want to know the customer satisfaction on each of the restaurants. We will scrape the customer's review from the wongnai website to see their opinion of each restaurant. <br>
Selenium library is used as the Wongnai website is developed using Java.

### Import necessary libraries

In [1]:
# -*- coding: utf-8 -*-
%pip install -q webdriver-manager
%pip install -q selenium numpy pandas 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver
import numpy as np
import time
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException
import pandas as pd
from tqdm import tqdm
import pickle
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


In [ ]:
# # Install and use webdriver-manager to get the correct chromedriver
# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [19]:
# Install undetected-chromedriver for bypassing Cloudflare
%pip -q install undetected-chromedriver

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Scrape the link of each restaurant

We will scrape all the restaurants in the first page of the restaurant.

In [20]:
import undetected_chromedriver as uc



driver = uc.Chrome(headless=True,use_subprocess=False)

driver.get('https://nowsecure.nl')

ModuleNotFoundError: No module named 'distutils'

In [4]:
# Specify the link wanted to scrape

link = 'https://www.wongnai.com/restaurants?categoryGroupId=9&regions=373'

In [6]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# Re-initialize the driver in case the previous session is closed
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

scrape_list = []

# We will scrape only first two pages of the wongnai website.
for i in np.arange(1,3,1):
    link_str = link+str(i)
    driver.get(link_str)
    scrape = driver.find_elements(By.CLASS_NAME, "bscrqm-0.evrsfu") # this class contain href element
    scrape_list.append([i.get_attribute("href") for i in scrape]) # get the href link and append into the list

In [23]:
# Remove duplicate

scrape_list = [list(set(scrape_list[i])) for i in range(len(scrape_list))]    

In [26]:
scrape_num = 0
for i in range(len(scrape_list)):
    scrape_num += len(scrape_list[i])
    
print("Total number of restaurants:", scrape_num)

Total number of restaurants: 0


### Data acquisition

As Wongnai website is created using Java. Some of the necessary tasks are required to perform such as: click more botton and unhide the hidden comments.

In [63]:
# List of list all comments, images of score and date of the comment

comments = []
images = []
dates = []
restaurants = []

In [ ]:
# Scraping Time!!

for i in tqdm(range(len(scrape_list))):    
    for j in tqdm(range(len(scrape_list[i]))):
        driver.get(scrape_list[i][j])
        
        # Make all the comments visible
        # while the more button is present, click it
        while True:
            try:
                driver.find_element(By.CLASS_NAME, "sc-bZQynM.d0pjph-1.jAZAn.fropwq-1.xsZYA").click() #click the more button
                time.sleep(3) # delay
            except NoSuchElementException: # stop when no click button
                break
            except ElementClickInterceptedException: # continue the loop even the error present (this error came from the more button is not yet loaded)
                continue
            
        # unhide the hidden comment    
        while True:
            try:
                driver.find_element(By.CLASS_NAME, "w0wgso-12.cVFAR").click() #click the more button
                time.sleep(3) # delay
            except NoSuchElementException: # stop when no click button
                break
            except ElementClickInterceptedException:
                continue
                
        # get only relevant table
        scrape_all = driver.find_elements(By.CLASS_NAME, "w0wgso-5.fqqMQC")
        scrape_all2 = driver.find_elements(By.CLASS_NAME, "sc-1ygfv81-3.glBSGy.sc-188xx7w-0.APLAw") # reviewer that come back to review the restaurant again
        scrape_name = driver.find_element(By.CLASS_NAME, "sc-1fjq89s-1.etVlxd").text
        
        comments_group = []
        score_group = []         
        date_group = []
        restaurant_name_group = []
        
        for k in tqdm(range(len(scrape_all))): 
            # scrape scores
            try: 
                score = scrape_all[k].find_element(By.CLASS_NAME, "w0wgso-7.iHbfIZ").find_element(By.TAG_NAME, 'img').get_attribute('src')
            except NoSuchElementException:
                score = "no score" # if no score being present, "no score" string is applied
            
            # scrape comment
            try:   
                text = scrape_all[k].find_element(By.CLASS_NAME, "w0wgso-2.tBGOX").text           
            # if error from no comment, input as no comment
            except NoSuchElementException:
                text = "no comment" # if no comment is written, "no comment" string is applied
            
            # scrape name already done above
            
            # scrape date    
            date = scrape_all[k].find_element(By.CLASS_NAME, "osf0bs-2.cIiaCV").text
            
            comments_group.append(text)
            score_group.append(score)
            date_group.append(date)
            restaurant_name_group.append(scrape_name)
            
        # scrape the comments and score for those reviewer that come back to review again
        for l in tqdm(range(len(scrape_all2))):
            try:   
                text = scrape_all2[l].text           
            # if error from no comment, input as no comment
            except NoSuchElementException:
                text = "no comment"
            
            try: 
                score = scrape_all2[l].find_element(By.CLASS_NAME, "sc-1ygfv81-0.ieRMWA").find_element(By.TAG_NAME, 'img').get_attribute('src')
            except NoSuchElementException:
                score = "no score"
        
            comments_group.append(text)
            score_group.append(score)
            date_group.append(date)
            restaurant_name_group.append(scrape_name)
        
        comments.append(comments_group)
        images.append(score_group)
        dates.append(date_group)
        restaurants.append(restaurant_name_group)









  0%|                                                                                            | 0/2 [00:00<?, ?it/s]








  0%|                                                                                           | 0/22 [00:00<?, ?it/s]









  0%|                                                                                          | 0/399 [00:00<?, ?it/s]









  0%|▏                                                                                 | 1/399 [00:00<00:40,  9.83it/s]









  1%|▍                                                                                 | 2/399 [00:00<00:41,  9.67it/s]









  1%|▌                                                                                 | 3/399 [00:00<00:40,  9.74it/s]









  1%|▊                                                                                 | 4/399 [00:00<00:40,  9.77it/s]









  1%|█                                                                                 | 5

 34%|██████████████████████████▊                                                     | 134/399 [00:14<00:30,  8.78it/s]









 34%|███████████████████████████                                                     | 135/399 [00:14<00:29,  8.87it/s]









 34%|███████████████████████████▎                                                    | 136/399 [00:14<00:29,  8.80it/s]









 34%|███████████████████████████▍                                                    | 137/399 [00:14<00:30,  8.71it/s]









 35%|███████████████████████████▋                                                    | 138/399 [00:14<00:29,  8.76it/s]









 35%|███████████████████████████▊                                                    | 139/399 [00:14<00:29,  8.89it/s]









 35%|████████████████████████████                                                    | 140/399 [00:15<00:29,  8.74it/s]









 35%|████████████████████████████▎                                                   | 141/399 [0

 65%|████████████████████████████████████████████████████▎                           | 261/399 [00:30<00:17,  8.06it/s]









 66%|████████████████████████████████████████████████████▌                           | 262/399 [00:30<00:17,  7.88it/s]









 66%|████████████████████████████████████████████████████▋                           | 263/399 [00:30<00:17,  7.65it/s]









 66%|████████████████████████████████████████████████████▉                           | 264/399 [00:30<00:18,  7.48it/s]









 66%|█████████████████████████████████████████████████████▏                          | 265/399 [00:30<00:16,  7.90it/s]









 67%|█████████████████████████████████████████████████████▎                          | 266/399 [00:30<00:19,  6.82it/s]









 67%|█████████████████████████████████████████████████████▌                          | 267/399 [00:31<00:18,  6.98it/s]









 67%|█████████████████████████████████████████████████████▋                          | 268/399 [0

 97%|█████████████████████████████████████████████████████████████████████████████▌  | 387/399 [00:45<00:01,  8.81it/s]









 97%|█████████████████████████████████████████████████████████████████████████████▊  | 388/399 [00:45<00:01,  8.85it/s]









 97%|█████████████████████████████████████████████████████████████████████████████▉  | 389/399 [00:45<00:01,  8.95it/s]









 98%|██████████████████████████████████████████████████████████████████████████████▏ | 390/399 [00:45<00:01,  8.95it/s]









 98%|██████████████████████████████████████████████████████████████████████████████▍ | 391/399 [00:45<00:00,  9.13it/s]









 98%|██████████████████████████████████████████████████████████████████████████████▌ | 392/399 [00:46<00:00,  9.29it/s]









 98%|██████████████████████████████████████████████████████████████████████████████▊ | 393/399 [00:46<00:00,  8.73it/s]









 99%|██████████████████████████████████████████████████████████████████████████████▉ | 394/399 [0

 25%|████████████████████▍                                                           | 115/451 [00:13<00:38,  8.70it/s]









 26%|████████████████████▌                                                           | 116/451 [00:13<00:38,  8.61it/s]









 26%|████████████████████▊                                                           | 117/451 [00:13<00:38,  8.69it/s]









 26%|████████████████████▉                                                           | 118/451 [00:13<00:37,  8.79it/s]









 26%|█████████████████████                                                           | 119/451 [00:13<00:38,  8.72it/s]









 27%|█████████████████████▎                                                          | 120/451 [00:13<00:37,  8.84it/s]









 27%|█████████████████████▍                                                          | 121/451 [00:13<00:37,  8.80it/s]









 27%|█████████████████████▋                                                          | 122/451 [0

...

 57%|█████████████████████████████████████████████▍                                  | 101/178 [00:10<00:08,  9.40it/s]









 57%|█████████████████████████████████████████████▊                                  | 102/178 [00:10<00:07,  9.52it/s]









 58%|██████████████████████████████████████████████▎                                 | 103/178 [00:10<00:07,  9.48it/s]









 58%|██████████████████████████████████████████████▋                                 | 104/178 [00:10<00:07,  9.42it/s]









 59%|███████████████████████████████████████████████▏                                | 105/178 [00:10<00:07,  9.54it/s]









 60%|███████████████████████████████████████████████▋                                | 106/178 [00:10<00:07,  9.49it/s]









 60%|████████████████████████████████████████████████                                | 107/178 [00:11<00:07,  9.27it/s]









 61%|████████████████████████████████████████████████▌                               | 108/178 [0

 56%|████████████████████████████████████████████▋                                   | 114/204 [00:11<00:09,  9.32it/s]









 57%|█████████████████████████████████████████████▍                                  | 116/204 [00:11<00:09,  9.54it/s]









 57%|█████████████████████████████████████████████▉                                  | 117/204 [00:11<00:09,  9.54it/s]









 58%|██████████████████████████████████████████████▎                                 | 118/204 [00:11<00:09,  9.41it/s]









 58%|██████████████████████████████████████████████▋                                 | 119/204 [00:12<00:09,  9.29it/s]









 59%|███████████████████████████████████████████████                                 | 120/204 [00:12<00:08,  9.39it/s]









 59%|███████████████████████████████████████████████▍                                | 121/204 [00:12<00:08,  9.39it/s]









 60%|███████████████████████████████████████████████▊                                | 122/204 [0

 51%|█████████████████████████████████████████▋                                       | 91/177 [00:09<00:08,  9.65it/s]









 52%|██████████████████████████████████████████                                       | 92/177 [00:09<00:08,  9.65it/s]









 53%|██████████████████████████████████████████▌                                      | 93/177 [00:09<00:09,  9.07it/s]









 53%|███████████████████████████████████████████                                      | 94/177 [00:09<00:08,  9.26it/s]









 54%|███████████████████████████████████████████▍                                     | 95/177 [00:09<00:08,  9.27it/s]









 54%|███████████████████████████████████████████▉                                     | 96/177 [00:09<00:08,  9.43it/s]









 55%|████████████████████████████████████████████▊                                    | 98/177 [00:10<00:08,  9.64it/s]









 56%|█████████████████████████████████████████████▏                                  | 100/177 [0

 68%|██████████████████████████████████████████████████████▊                          | 96/142 [00:09<00:04,  9.97it/s]









 69%|███████████████████████████████████████████████████████▉                         | 98/142 [00:09<00:04, 10.11it/s]









 70%|████████████████████████████████████████████████████████▎                       | 100/142 [00:10<00:04, 10.32it/s]









 72%|█████████████████████████████████████████████████████████▍                      | 102/142 [00:10<00:03, 10.13it/s]









 73%|██████████████████████████████████████████████████████████▌                     | 104/142 [00:10<00:03, 10.21it/s]









 75%|███████████████████████████████████████████████████████████▋                    | 106/142 [00:10<00:03, 10.24it/s]









 76%|████████████████████████████████████████████████████████████▊                   | 108/142 [00:10<00:03, 10.25it/s]









 77%|█████████████████████████████████████████████████████████████▉                  | 110/142 [0

 84%|███████████████████████████████████████████████████████████████████▉             | 88/105 [00:08<00:01, 11.29it/s]









 86%|█████████████████████████████████████████████████████████████████████▍           | 90/105 [00:08<00:01,  9.66it/s]









 88%|██████████████████████████████████████████████████████████████████████▉          | 92/105 [00:09<00:01, 10.13it/s]









 90%|████████████████████████████████████████████████████████████████████████▌        | 94/105 [00:09<00:01, 10.67it/s]









 91%|██████████████████████████████████████████████████████████████████████████       | 96/105 [00:09<00:00, 10.48it/s]









 93%|███████████████████████████████████████████████████████████████████████████▌     | 98/105 [00:09<00:00, 10.71it/s]









 95%|████████████████████████████████████████████████████████████████████████████▏   | 100/105 [00:09<00:00, 10.62it/s]









 97%|█████████████████████████████████████████████████████████████████████████████▋  | 102/105 [0

### Data Preparation

In [69]:
with open(r'./data/wongnai_data.pkl', 'wb') as f:
    pickle.dump(comments,f)
    
with open(r'./data/score_data.pkl', 'wb') as f:
    pickle.dump(images,f)

with open(r'./data/date_data.pkl', 'wb') as f:
    pickle.dump(dates,f)
    
with open(r'./data/restaurants_name.pkl', 'wb') as f:
    pickle.dump(restaurants,f)

In [70]:
name = pd.DataFrame(restaurants).T
data = pd.DataFrame(comments).T   
score = pd.DataFrame(images).T
date = pd.DataFrame(dates).T

In [77]:
# Find unqie image score
all_img = score.T.copy()
all_img = list(set(all_img[0].dropna()))

# Assign score
img_dict = {"https://static2.wongnai.com/static2/images/2tFE9yR.png": 3,
            "https://static2.wongnai.com/static2/images/1xWrjob.png": 5,
            "https://static2.wongnai.com/static2/images/j1U4KcR.png": 4,
            "https://static2.wongnai.com/static2/images/4VXmdfq.png": 1,
            "https://static2.wongnai.com/static2/images/1ELyvQs.png": 2}


In [78]:
# Maping image with score

score = score.apply(lambda x: x.map(img_dict))

### Data Preprocess

In [145]:
# Join Dataframe for restaurant 1

def create_data(restaurant_index = 0):
    df = pd.DataFrame([name[restaurant_index], data[restaurant_index], score[restaurant_index], date[restaurant_index]]).T
    
    # Rename column
    col_name = ["restaurant_name", "comments", "score", "date"]
    df.columns = col_name
    return(df) 

In [146]:
# Create data of restaurant 1
df1 = create_data(1)
df1.head()

,restaurant_name,comments,score,date
0,ข้าวหมูแดงสีมรกต (Si Morakot) Soi Su Korn 1,เป็นอีกร้านที่อยู่ใน wishlist อยู่นานว่าต้องไป...,4,เมื่อ 2 อาทิตย์ที่แล้ว
1,ข้าวหมูแดงสีมรกต (Si Morakot) Soi Su Korn 1,ข้าวหมูแดงและข้าวหมูกรอบสีมรกตเป็นร้านที่โด่งด...,3,เมื่อ 3 วันที่แล้ว
2,ข้าวหมูแดงสีมรกต (Si Morakot) Soi Su Korn 1,ข้าวหมูแดงสีมรกต\n.\n..\nอยากหม่ำข้าวหมูแดง แว...,4,เมื่อ 2 เดือนที่แล้ว
3,ข้าวหมูแดงสีมรกต (Si Morakot) Soi Su Korn 1,วันอาทิตย์เราก็ต้องทำงาน ต้องหาอะไรเด็ดๆมากินแ...,5,เมื่อ 2 เดือนที่แล้ว
4,ข้าวหมูแดงสีมรกต (Si Morakot) Soi Su Korn 1,สั่งข้าวหมูแดงหมูกรอบเพิ่มไข่ยางมะตูมพิเศษจัดห...,3,เมื่อ 3 เดือนที่แล้ว


In [147]:
# count null vaue

df1.isnull().sum()

restaurant_name    98
comments           98
score              98
date               98
dtype: int64

In [148]:
# count no comment

df1[df1.comments == "no comment"].count()

restaurant_name    92
comments           92
score              92
date               92
dtype: int64

In [150]:
# count no score

df1[df1.score == "no score"].count()

restaurant_name    0
comments           0
score              0
date               0
dtype: int64

In [151]:
df1.shape

(549, 4)

In [152]:
df1.dropna(how="all").shape

(451, 4)

In [154]:
df1.dropna(how="all", inplace=True)
df1.tail()

,restaurant_name,comments,score,date
446,ข้าวหมูแดงสีมรกต (Si Morakot) Soi Su Korn 1,กรอบมาก อร่อย,5,14 ส.ค. 2014
447,ข้าวหมูแดงสีมรกต (Si Morakot) Soi Su Korn 1,ร้านนี้เป็นร้านขายข้าวหมูแดงที่อร่อยมาก จุดเด่...,5,15 พ.ค. 2013
448,ข้าวหมูแดงสีมรกต (Si Morakot) Soi Su Korn 1,อร่อยมากกกก,5,4 พ.ค. 2012
449,ข้าวหมูแดงสีมรกต (Si Morakot) Soi Su Korn 1,ปกติไม่ชอบกินหมู แต่ร้านนี้กินหมดเกลี้ยงเลย,5,4 เม.ย. 2012
450,ข้าวหมูแดงสีมรกต (Si Morakot) Soi Su Korn 1,no comment,5,18 ก.ค. 2011
